그러므로 이제 그리스도 예수 안에 있는 자에게는 결코 정죄함이 없나니 이는 그리스도 예수 안에 있는 생명의 성령의 법이 죄와 사망의 법에서 너를 해방하였음이라 (롬8:1-2)

-------
<center><img src="https://github.com/idebtor/DSpy/blob/6b3676c3de29f85d2e7f30679676d1fd96d88b17/images/WelcomeToDataStructures.jpg?raw=true" width=1000></center>

__NOTE:__ The following materials have been compiled and adapted from the numerous sources including my own. Please help me to keep this tutorial up-to-date by reporting any issues or questions. Send any comments or criticisms to `idebtor@gmail.com` Your assistances and comments will be appreciated.

--------

<b style="font-size:30px">  Chapter 7-4 Binary Search Tree </b>

: Tree(트리)라는 자료형이란 무엇이며 어떻게 활용되는지, 그리고 이를 효과적으로 활용할 수 있는 트리 알고리즘을 배웁니다.  

--------------
제 7장에서는 트리(Tree) 자료형과 그에 활용되는 여러 트리 알고리즘(Tree Algorithm)에 대해 알아봅니다.

# Binary Search Tree

이진 검색 트리는 상위보다 작은 키가 왼쪽 subtree에 있고 상위보다 큰 키가 오른쪽 subtree에 있는 속성에 의존합니다. 왼쪽 subtree의 모든 키는 root의 키보다 작고, 오른쪽 subtree의 모든 키는 root보다 큽니다.

<center><img src="https://github.com/idebtor/DSpy/blob/main/images/chap6/ch6-13.png?raw=true" width="500" title="Figure 1. A Simple Binary Search Tree">
<center>그림 1. 간단한 이진 검색 트리의 예<center><br>

위의 이진 검색 트리 예시에서 루트(root)인 70을 기준으로 70보다 작으면 왼쪽 노드에, 70보다 크면 오른쪽 노드에 위치하게 됩니다. 

이진 검색 트리에서 노드를 삽입하는 것을 애니메이션으로 표현하면 다음과 같습니다.

<center><img src="https://github.com/idebtor/DSpy/blob/main/images/chap6/ch6-13.gif?raw=true" width="500" title="Figure 2. Binary Search Tree Insert">
<center>그림 2. 이진 검색 트리에 노드 삽입 과정<center><br>

이진 검색 트리에 대한 이해를 마쳤다면, 본격적으로 이진 검색 트리를 코딩해보도록 하겠습니다.  

## 노드(Node) 클래스 정의와 초기화 (Review)

이진 탐색 트리를 구현하려면 먼저 그 재료가 되는 `Node` 클래스를 정의해야 합니다. Node 클래스는 노드값(self.key)과 좌/우 노드(self.left, self.right), 총 세 개의 속성을 갖습니다. 초기화할 때는 데이터 값만 주어지고 좌우 노드가 비어있습니다. 이와 같이 인진 탐색트리도 이진트리의 의 일종이므로, 이진트리와 특성이 같습니다. 

그러므로, 우리는 이제 아래와 같이 이미 앞 단원에서 정의한 node.py 모듈을 사용하도록 합니다. 

In [ ]:
class Node():
    def __init__(self, key, left = None, right = None):
        self.key = key
        self.left = left
        self.right = right

    def __repr__(self):
        return "Node({})".format(self.key)
  
    def __iter__(self):
        curr_nodes = [self]
        while len(curr_nodes) > 0:
            next_nodes = []
            for node in curr_nodes:
                yield node
                if node.left is not None:
                    next_nodes.append(node.left)
                if node.right is not None:
                    next_nodes.append(node.right)
            curr_nodes = next_nodes

if __name__ == '__main__':
    root = Node(1)
    root.left = Node(2)
    root.right = Node(3)
    print(root)
    print(list(root))

## BST 클래스 정의와 초기화

`BST`(Binary Search Tree) 클래스에는 이진 검색 트리의 root인 Node에 대한 참조가 있습니다. 트리가 빈 것인지 아닌지 BST 자체에서 확인할 수 있습니다.  대부분의 트리 연산은 BST 클래스가 처리합니다. 

이제 Node 클래스를 사용해 이진 탐색 트리 클래스인 BST의 기본 클래스를 만들 수 있습니다. 다음은 완성되지 않은 간단한 BST 클래스 코드입니다. 

In [ ]:
from node import Node

class BST:
    def __init__(self, key = None, left = None, right = None):
        if key is None:
            self.root = None
        else:
            self.root = Node(key, left, right)
    
    def delTree(self):
        self.root = None   # garbage collector will do this for us.
        
    def __str__(self):
        return self.root.__str__()
    
    def __iter__(self):
        """iterate through the nodes in the binary tree in level-order"""
        return self.root.__iter__()
        
    def __len__(self):
        return self.root.__len__()
    
    def len(self):
        return self.__len__()

이제 여기에 노드를 추가, 삭제, 탐색할 수 있도록 `add(), delelte(), contains()` 메서드를 추가해야 합니다.

## 삽입 메소드 - `add()`
BST에 add() 메서드를 구현하여 트리에 새 노드를 추가할 수 있도록 합니다. 재귀를 사용하면 쉽게 구현할 수 있습니다. 새로 추가할 노드의 값을 현재 노드의 값과 비교하여 왼쪽/오른쪽 중 알맞은 위치로 노드를 옮겨가며 삽입 위치를 찾아갑니다.


In [ ]:
class BST:
    
    ...
    
    def add(self, key):
        self.root = self._add(self.root, key)
        return self.root is not None

    def _add(self, node, key):
        if node is None:
            node = Node(key)
        else:
            if key < node.key:
                node.left = self._add(node.left, key)
            elif key > node.key:
                node.right = self._add(node.right, key)
            else:
                pass
        return node

`_add`는 재귀를 위한 일종의 도우미 함수이며, 알고리즘은 다음과 같습니다.  

- 트리의 root에서 시작하여 새 키를 현재 노드의 키와 비교하여 이진 트리를 검색합니다. 새 키가 현재 노드보다 작으면 왼쪽 subtree를 검색합니다. 새 키가 현재 노드보다 크면 오른쪽 subtree를 검색합니다.   
- 검색할 왼쪽(또는 오른쪽) 하위 항목이 없으면 트리에서 새 노드를 설치할 위치를 찾은 것입니다.   
- 트리에 노드를 추가하려면 새 Node 개체를 생성하고 이전 단계에서 발견된 지점에 개체를 삽입합니다.  

그런데, 삽입 구현에서 한 가지 중요한 문제는 중복 키가 제대로 처리되지 않는다는 것입니다. 여기서는 중복 키가 들어오면, 아무것도 하지 않습니다.  


## 탐색 메소드 - `get(), __getitem__(), __contains()__, contains()`
원하는 값의 존재유무를 확인할 수 있도록 메서드를 구현합니다. 이 역시 재귀와 값의 대소관계 비교를 통해 쉽게 구현할 수 있습니다.

In [ ]:
class BST:
    ...
    def get(self, key):
        return self._get(self.root, key)

    def _get(self, root, key):
        if root is None or root.key == key:
            return root is not None
        elif key < root.key:
            return _get(root.left, key)
        elif key > root.key:
            return _get(root.right, key)

    def __getitem__(self, key):
        return self.get(key).key
    
    def __contains__(self, key):
        return bool(self.get(key))
    
    def contains(self, key):
        return self.__contains__(key)

## 삭제 메소드

삭제는 비교적 까다롭습니다. 왜 그럴까요? 다음과 같이 세 가지의 경우가 있기 때문입니다. 

__Case 1:__ (자식이 없는 경우) 삭제할 노드에게 자식이 없으면, 자기 자신을 삭제하고, 자신과 연결되어 있는 상위 노드에 None을 반환합니다. 

__Case 2:__ (자식이 하나있는 경우) 삭제할 노드에게 자식이 하나인 경우엔, 자기 자신을 삭제하고 즉 자신을 자식의 노드 정보로 갱신하고 자신을 반환합니다.   

__Case 3:__ (자식이 둘있는 경우) 삭제할 노드의 자식이 둘이 있을 때는 어느 자식 노드를 어떻게 가져와야 할까요? 삭제할 노드의 양쪽에 자식이 있으므로, 왼쪽이나 오른쪽에서 하나를 취할 수 있는 두 가지 방법이 있습니다. 

- 이제 왼쪽 서브 트리(A)와 오른쪽 서브 트리(B)를 각각 A, B라고 부른다고 하면, 
- 서브트리 A에서 가장 큰 자손(predecessor)을 가져오면 됩니다. 그런데, 이는 서브트리 A에서 가장 오른쪽에 위치한 자손(predecessor)이며, 이 원소는 A의 모든 원소들보다 크며, B 서브 트리보다는 작기 때문에 삭제할 노드를 대신할 키를 가지고 있습니다. (이것이 이진 탐색 트리의 특성입니다). 또 다른 방법은 
- 서브트리 B에서 가장 작은 자손(successor)을 가져오면 됩니다. 그런데, 이는 원소는 서브트리 A의 모든 원소들보다 크면서, 서브트리 B의 나머지 원소보다는 작기 때문입니다. (이것이 이진 탐색 트리의 특성입니다).

__Notes:__
- 이러한 predecessor 혹은 successor 노드는 자식이 두 개가 있을 수 없으며, 자식은 언제나 하나이거나 없습니다. 
- 삭제할 노드의 왼쪽과 오른쪽의 서브트리의 높이를 계산하여, 높이를 낮출 수 있는 방향으로 삭제를 진행하는 것이 균형 잡힌 트리를 만들어 나가는데 도움이 됩니다. 이러한 방법으로 구현하는 것은 코딩 연습문제로 주어집니다. 

In [ ]:
class BST(object):
    ...
    def delete(self, key):
        self.root = self._delete(self.root, key)

    def _delete(self, node, key):
        if node is None: return node

        if key < node.key:
            node.left = self._delete(node.left, key)
        elif key > node.key:
            node.right = self._delete(node.right, key)
        else:   #  key == node.key # the node to delete is found
            if node.left and node.right:    # two children case 
                # replace the node to the leftmost of node.right
                # find the successor(child) and its parent
                parent, child = node, node.right
                while child.left is not None:
                    parent, child = child, child.left
                
                child.left = node.left   
                if parent != node:   
                    parent.left = child.right
                    child.right = node.right
                node = child
            elif node.left or node.right:   # one child
                node = node.left or node.right
            else:                           # no child
                node = None
        return node

삭제할 키가 들어 있는 노드를 찾은 경우 고려해야 할 세 가지 사례가 있습니다.

1. 삭제할 노드에 하위 노드가 없습니다   
1. 삭제할 노드에 하위 노드가 하나만 있습니다(그림 4 참조).   
1. 삭제할 노드에 두 개의 하위 노드가 있습니다(그림 5 참조).   

첫 번째 경우는 현재 노드에 하위 노드가 없는 경우 노드를 삭제하고 상위 노드에서 이 노드에 대한 참조를 제거하면 됩니다.
두 번째 경우는 노드에 __한 개의__ 하위 항목만 있는 경우, 그 한 개의 하위 항목을 상위 즉 현재의 노드 위치로 올려주면 됩니다. 

```
            elif node.left or node.right:   # one child
                node = node.left or node.right
            else:                           # no child
                node = None

```

<center><img src="https://github.com/idebtor/DSpy/blob/main/images/chap6/ch6-15.png?raw=true" width="900" title="Figure 4. Deleting Node 25, a Node That has a Single Child">
<center>그림 4. 자식 노드가 하나 있는 노드(25) 제거<center><br>

__Case 3:__ 세 번째 사례는 노드에 두 개의 하위 노드들이 있는 경우 노드 대신 한 개의 하위 항목을 승격할 수 없습니다. __그러나__ 트리에서 삭제하려고 예약된 노드를 대체하는 데 사용할 수 있는 노드를 검색할 수 있습니다. 기존 왼쪽 및 오른쪽 subtree 모두에 대해 이진 검색 트리 관계를 보존할 노드가 필요합니다. __이렇게 할 노드는 트리에서 다음으로 큰 키를 가진 노드입니다.__ 우리는 이 노드를 후계자(__successor__)라고 부르며, 곧 후계자를 찾는 방법을 모색할 것입니다. 특히, 후계자에게 자녀가 한 명 이상 없을 것이 되어 있기 때문에, 이미 구현한 두 가지 삭제 사례를 사용하여 후계자를 삭제하되, 후계자의 값들 즉 key를 예정한 노드에 갱신하면 됩니다. 

결론적으로, 삭제하려고 찾아간 현재 노드에 두 개의 하위 노드가 있으면, 현재 노드의 successor를 찾아가서 successor의 값(key)으로 현재 노드의 값(key)를 갱신하고, 이제 successor를 삭제합니다. 단 successor를 삭제하는 방법은 이미 앞에서 구현한 하위 노드가 없거나, 하나인 경우에 해당합니다.  이는 아래와 같이 요약할 수 있습니다. 

아래 코드에서 successor를 찾은 후, 삭제할 노드를 갱신할 때 두 가지 방법을 제시합니다. 
- 첫 번째 방법은 삭제할 노드가 가지고 있는 링크를 child 노드에 (key, left, right)에 가져와 설정하고 있습니다.  
- 두 번째 방법은 삭제할 노드가 가지고 있는 링크를 그대로 살리고, child 노드의 key만 가져와 삭제할 노드에 복사하고, successor의 child 링크 정보를 parent에 링크하는 방법입니다. 

여기 제시된 구현은 반복문을 사용하여 구현하였지만, 재귀를 사용하여 구현하는 방법이 있습니다. successor (or predecessor)를 구하여, 삭제할 노드의 값(key)를 successor의 값으로 갱신 한 후에, 서브트리에서 successor를 삭제하도록 재귀적으로 삭제 `_delete()` 함수를 다시 호출하는 방법입니다. 이 방법은 강의에서 설명하고 있으며, 코딩 연습 문제로 주어집니다. 

```
if node.left and node.right:       # two children
    # replace the node to the leftmost of node.right
    # find the successor (child) and its parent 
    parent, child = node, node.right
    while child.left is not None:
        parent, child = child, child.left   

    # replace the node's contents(key, left, right) with successor's
    child.left = node.left         # keep the old link for (2)
    if parent != node:             # if the successor is not immediate child
        parent.left = child.right  # (9).left = (7).right, to keep successor's child if any
        child.right = node.right   # to keep the old link for (11)
    node = child
```

```
    # alternatively, keep the node's links, but replace the key only with successor's
    node.key = child.key            # replace the node's key with successor only
    if parent == node:              # connect successor's child to its parent
        parent.right = child.right  # successor has one right child IF ANY. 
    else:                           # successor's parent exists at left or right
        parent.left = child.right
```

<center><img src="https://github.com/idebtor/DSpy/blob/main/images/chap6/ch6-16.png?raw=true" width="900" title="Figure 5. Deleting Node 5, a Node with Two Children">
<center>그림 5. 자식 노드가 두개 있는 노드(5) 제거<center><br>

후계자(successor)를 찾는 코드는 다음과 같습니다. 이 코드는 이진 검색 트리의 동일한 속성을 사용하여 오른쪽 subtree에서 가장 작은 값의 노드 즉 오른쪽 subtree에서 최소값 key를 가진 노드를 찾는 작업입니다.  

예를 들면, 다음 코드는 위의 그림에서, node = 5에서 시작하여, 결국 child 노드 7을 찾게 됩니다. 

```
                parent, child = node, node.right
                while child.left is not None:
                    parent, child = child, child.left
```

그러면 이제 전체 코드를 정리해 보겠습니다.

In [ ]:
#%%writefile bst.py 

from collections.abc import Iterable
from dspy.node import Node

class BST:
    def __init__(self, keys = None, left = None, right = None):
        if keys is None:
            self.root = None
        else:
            if isinstance(keys, Iterable): 
                self.root = Node(keys[0], left, right)
                for key in keys[1:]:
                    self.add(key) 
            else:
                self.root = Node(keys, left, right)
    
    def delTree(self):
        self.root = None   # garbage collector will do this for us.
        
    def __str__(self):
        return self.root.__str__()
    
    def __iter__(self):
        """iterate through the nodes in the binary tree in level-order"""
        return self.root.__iter__()
        
    def __len__(self):
        if self.root is None: return 0
        return self.root.__len__()
    
    def len(self):
        return self.__len__()
       
    def add(self, key):
        self.root = self._add(self.root, key)
        return self.root is not None

    def _add(self, node, key):
        if node is None:
            node = Node(key)
        else:
            if key < node.key:
                node.left = self._add(node.left, key)
            elif key > node.key:
                node.right = self._add(node.right, key)
            else:
                pass
        return node
    
    def get(self, key):
        return self._get(self.root, key)

    def _get(self, root, key):
        if root is None or root.key == key:
            return root is not None
        elif key < root.key:
            return self._get(root.left, key)
        elif key > root.key:
            return self._get(root.right, key)

    def __getitem__(self, key):
        return self.get(key)
    
    def delete(self, key):
        self.root = self._delete(self.root, key)
    
    def _delete(self, node, key):
        if node is None: return node

        if key < node.key:
            node.left = self._delete(node.left, key)
        elif key > node.key:
            node.right = self._delete(node.right, key)
        else:    #  key == node.key # the node to delete is found
            if node.left and node.right:
                # replace the node to the leftmost of node.right
                parent, child = node, node.right
                while child.left is not None:
                    parent, child = child, child.left
                    
                # replace the node's contents only(key, left, right)
                child.left = node.left
                if parent != node:
                    parent.left = child.right
                    child.right = node.right
                node = child
            elif node.left or node.right:
                node = node.left or node.right
            else:
                node = None
        return node
        
if __name__ == '__main__':
    bst = BST()
    for x in [17, 5, 25, 35, 2, 11, 29, 39, 9, 16, 7, 8]: 
        bst.add(x)
    print(bst)

    for x in [25, 5, 17]:
        print(f'delete({x})')
        bst.delete(x)
        print(bst)

# Summary

이번 장에서는 이진 탐색 트리를 분석하고 평가하는 방법을 배웠습니다. 

이진 트리의 좌우 균형이 맞으면 탐색/삽입/삭제의 시간복잡도가 $O(\log n)$입니다. 그러나 이진 탐색 트리는 정렬된 데이터에 취약합니다. 오름차순이든 내림차순이든 정렬된 데이터가 입력되면 한쪽으로 치우친 (skewed) 트리가 만들어집니다. 이 때, 최악의 경우 모든 데이터를 살펴야 할 수도 있어 시간복잡도가 $O(n)$이 됩니다.

# Exercises

## Node class definition Testing

다음은 BST 클래스에서 사용하는 Node 클래스를 정의한 것입니다. 노드의 key값이 str 타입일지라도 tree를 그려낼 수 있는 코드가 포함되어 있습니다. 노드 클래스를 간단히 테스트할 수 있도록(BST를 사용하지 않고), __Expected Output:__ 처럼 출력되도록 테스트 코드를 작성하십시오. root노드를 만들고, 그 다음 child들을 하나씩 연결해야 합니다. 

__Expected Output:__
```
[Node(99), Node(A), Node(B), Node(F)]
len:  4
level-order by __iter__: 
99
A
B
F

  99
 /  \
A    B
      \
       F
```

In [ ]:
from dspy.node import Node

if __name__ == '__main__':
    # your code here
    pass

## BST Class Definition

다음과 같이 __Sample Run:__ 의 결과처럼, BST 클래스를 테스트할 수 있도록 필요한 메소드들을 추가하십시오. 다만, 필요한 메소드는 Node가 아니라 BST 클래스에 추가하십시오. 그러나, Node에 있는 메소드들을 적극 활용해야 합니다. 그러한 메소드를 찾아내는 것도 이 문제에서 요구하는 것입니다. 

__Sample Run:__ 
```
if __name__ == '__main__':
    print("build 'faith' tree")
    bst = BST('faith')
    print(bst)
    print('length:', len(bst))
    print('height:', bst.height())
    print('  size:', bst.size())

    print('bst contains("a")', bst.contains('a'))             # True
    print('bst contains("b")', bst.contains('b'))             # False

    for x in ['a', 'b']:
        print(f'{x} in bst? ', x in bst)
    print(bst)

    # del 
    print("\ndel bst['f'], bst['i']")
    del bst['f']
    del bst['i']
    print(bst)
    
    print("\ndel all nodes")
    for x in bst:
        del bst[x.key]
    print(bst)
    print('length:', bst.len())
    print('height:', bst.height())  
```

__Expected Output:__
```
build 'faith' tree

  f__
 /   \
a     i
     / \
    h   t

length: 5
height: 2
  size: 5
bst contains("a") True
bst contains("b") False
a in bst?  True
b in bst?  False

  f__
 /   \
a     i
     / \
    h   t


del bst['f'], bst['i']

  h
 / \
a   t

del all nodes
None
length: 0
height: -1
```

In [ ]:
#%%writefile bst.py 

from collections.abc import Iterable
from dspy.node import Node

class BST:
    def __init__(self, keys = None, left = None, right = None):
        if keys is None:
            self.root = None
        else:
            if isinstance(keys, Iterable): 
                self.root = Node(keys[0], left, right)
                for key in keys[1:]:
                    self.add(key) 
            else:
                self.root = Node(keys, left, right)
    
    def delTree(self):
        self.root = None   # garbage collector will do this for us.
        
    def __str__(self):
        return self.root.__str__()
    
    def __iter__(self):
        """iterate through the nodes in the binary tree in level-order"""
        return self.root.__iter__()
        
    def __len__(self):
        if self.root is None: return 0
        return self.root.__len__()
    
    def len(self):
        return self.__len__()
       
    def add(self, key):
        self.root = self._add(self.root, key)
        return self.root is not None

    def _add(self, node, key):
        if node is None:
            node = Node(key)
        else:
            if key < node.key:
                node.left = self._add(node.left, key)
            elif key > node.key:
                node.right = self._add(node.right, key)
            else:
                pass
        return node
    
    def get(self, key):
        return self._get(self.root, key)

    def _get(self, root, key):
        if root is None: return None
    
        if key < root.key:
            return self._get(root.left, key)
        elif key > root.key:
            return self._get(root.right, key)
        else: # found it, return the node
            return root

    def __getitem__(self, key):
        return self.get(key)
    
    def delete(self, key):
        self.root = self._delete(self.root, key)
    
    def _delete(self, node, key):
        if node is None: return node

        if key < node.key:
            node.left = self._delete(node.left, key)
        elif key > node.key:
            node.right = self._delete(node.right, key)
        else:    #  key == node.key # the node to delete is found
            if node.left and node.right:
                # replace the node to the leftmost of node.right
                parent, child = node, node.right
                while child.left is not None:
                    parent, child = child, child.left
                    
                # replace the node's contents only(key, left, right)
                child.left = node.left
                if parent != node:
                    parent.left = child.right
                    child.right = node.right
                node = child
            elif node.left or node.right:
                node = node.left or node.right
            else:
                node = None
        return node

    #################### your code here #################################
    
```
if __name__ == '__main__':
    print("build 'faith' tree")
    bst = BST('faith')
    print(bst)
    print('length:', len(bst))
    print('height:', bst.height())
    print('  size:', bst.size())

    print('bst contains("a")', bst.contains('a'))             # True
    print('bst contains("b")', bst.contains('b'))             # False

    for x in ['a', 'b']:
        print(f'{x} in bst? ', x in bst)
    print(bst)

    # del 
    print("\ndel bst['f'], bst['i']")
    del bst['f']
    del bst['i']
    print(bst)
    
    print("\ndel all nodes")
    for x in bst:
        del bst[x.key]
    print(bst)
    print('length:', bst.len())
    print('height:', bst.height())  
```

## Add `height()` and `_height()` methods

- `height(self)` method simply calls `_height()` with `self.root`.
- `_height(self, node)` method recursively calls `_height(self, node)` and returns its height at the `node` of the tree.

위의 코드를 bst.py에 저장하고, 이를 아래와 같이 import하여 테스트합니다. 테스트 중에 위의 소스 코드를 수정한다면, `Restart Kernel`을 실행하여 bst 모듈을 다시 import하고 테스트를 진행하십시오. 

__Sample Run:__
```
from bst import BST

if __name__ == '__main__':
    print("build 'faith' tree")
    bst = BST('faith')
    print(bst)
    print('height: ', bst.height())
    for x in 'faith':
        print(f'height at {x}:', bst._height(bst.get(x)))
```

___Expected Output:__
```
build 'faith' tree

  f__
 /   \
a     i
     / \
    h   t

height:  2
height at f: 2
height at a: 0
height at i: 1
height at t: 0
height at h: 0
```

In [ ]:
from bst import BST

if __name__ == '__main__':
    print("build 'faith' tree")
    bst = BST('faith')
    print(bst)
    print('height: ', bst.height())
    for x in 'faith':
        print(f'height at {x}:', bst._height(bst.get(x)))

## Add `isBST()` method 

이진 탐색트리는 다음과 같이 조건을 만족해야 합니다. 
- 모든 노드는 유일한 키를 갖는다.
- 왼쪽 서브트리의 키들은 루트의 키보다 작고, 오른쪽 서브트리의 키들은 루트의 키보다 크다.
- 왼쪽과 오른쪽 서브트리도 이진탐색트리이다.

본 주제와 관련된 강의와 아래 그림을 참조하여 `isBST()`와 `_isBST()` 메소드를 구현하십시오. 

<center> <img src="https://github.com/idebtor/DSpy/blob/main/images/chap6/exercise-isBST.png?raw=true" width="800"> <center><br>
<center>그림 1. 이진 탐색트리 체크하기<center><br>
    
이 문제는 다음 `_delete()`관련 테스트 코드를 함께 사용하여 구현하고 테스트할 수 있도록 합니다. 

## Redefine `_delete()` two children case

지금까지 제공된 `_delete()` 메소드는 반복문을 사용하여 successor와 그의 parent 노드를 찾는 방식으로 구현하였습니다.  상당히 readability가 낮은 (난해한) 코드뿐 만 아니라, 무조건 successor를 삭제하는 방식입니다.

여기서는 좀 더 readable한 코드를 작성하고, 또한 삭제할 노드에서 볼 때, tree height가 큰 쪽의 노드를 찾아 대체하는 방법으로 구현함으로 tree를 balance를 추구하는 방향으로 코드를 수정하고자 합니다. 
 
- 삭제할 노드를 발견하면, 그 노드의 왼쪽과 오른쪽 서브트리의 height를 각각 계산합니다. 
- 왼쪽 서브트리의 height가 더 크다면, 왼쪽 서브트리에서 predecessor를 찾아서 노드 삭제를 진행하고, 오른쪽 서브트리의 height가 더 크거나 같다면, successor를 사용합니다.  
- 삭제할 노드의 key 값을 서브트리의 predecessor or successor의 key값으로 갱신합니다. 
- 이제 해당 서브트리의 predecessor or successor 노드를 삭제하도록 재귀적(recursively)으로 `_delete()`메소드를 호출합니다.  (자세한 설명은 강의/자료를 참고하십시오.)
- 이를 위해 `minimum(), _minimum(), maximum(), _maximum(), predecessor(), _predecessor(), successor(), successor()` 메소드를 정의하십시오. 

__Sample Run:__

```
    bst = BST([17, 5, 25, 35, 2, 11, 29, 39, 9, 15, 7, 8])
    print(bst)
    
    for _ in range(5):
        print(f'delete(root)')
        bst.delete()
        print(bst)
        print('still a BST?', bst.isBST())
    
    print('\nChange "root.left.right" to node(12) and test it for isBST()')
    bst.root.left.right.key = 12
    print(bst)
    print('still a BST?', bst.isBST())
    print(bst)    
```

__Expected Output:__

`delete()`로 삭제된 그래프는 아래와 같습니다. 

<center> <img src="https://github.com/idebtor/DSpy/blob/main/images/chap6/ch6-exercise.png?raw=true" width="800" title="Delete root 5 times consecutively"> <center><br>
<center>그림 2. Root 노드를 연속적으로 5회 삭제할 경우<center><br>

마지막 부분에 노드 값을 수정하고 테스트하는 경우입니다. 
```
Change node(9) to node(12) and test it for isBST()

    _______11
   /         \
  5____       25___
 /     \           \
2     __12         _35
     /            /   \
    7            29    39
     \
      8

still a BST? False

    _______11
   /         \
  5____       25___
 /     \           \
2     __12         _35
     /            /   \
    7            29    39
     \
      8
``` 

In [ ]:
# your BST code here


if __name__ == '__main__':
    bst = BST([17, 5, 25, 35, 2, 11, 29, 39, 9, 15, 7, 8])
    print(bst)
    
    for _ in range(5):
        print(f'delete(root)')
        bst.delete()
        print(bst)
        print('still a BST?', bst.isBST())
    
    print('\nChange "root.left.right" to node(12) and test it for isBST()')
    bst.root.left.right.key = 12
    print(bst)
    print('still a BST?', bst.isBST())
    print(bst)

# Tree 시각화 코드를 포함한 Node 클래스 정의

다음은 tree를 시각화하는 코드를 포함 Node 클래스이며, dspy/node.py에 저장되어 있습니다.

In [ ]:
# %%writefile dspy/node.py
class Node():
    def __init__(self, key, left = None, right = None):
        self.key = key
        self.left = left
        self.right = right
        
    def __repr__(self):
        return "Node({})".format(self.key)
    
    def __len__(self):
        return sum(1 for _ in iter(self))

    def __iter__(self):
        """iterate through the nodes in the binary tree in level-order"""
        curr_nodes = [self]

        while len(curr_nodes) > 0:
            next_nodes = []
            for node in curr_nodes:
                yield node
                if node.left is not None:
                    next_nodes.append(node.left)
                if node.right is not None:
                    next_nodes.append(node.right)
            curr_nodes = next_nodes
    
    def __str__(self):
        lines = self._build_tree_string(0, False, "-")[0]
        return "\n" + "\n".join((line.rstrip() for line in lines))
    
    def _build_tree_string(root, curr_index, include_index = False, delimiter = "-"):
        if root is None:
            return [], 0, 0, 0

        line1 = []
        line2 = []
        if include_index:
            node_repr = "{}{}{}".format(curr_index, delimiter, root.key)
        else:
            node_repr = str(root.key)

        new_root_width = gap_size = len(node_repr)

        # Get the left and right sub-boxes, their widths, and root repr positions
        l_box, l_box_width, l_root_start, l_root_end = Node._build_tree_string(
            root.left, 2 * curr_index + 1, include_index, delimiter
        )
        r_box, r_box_width, r_root_start, r_root_end = Node._build_tree_string(
            root.right, 2 * curr_index + 2, include_index, delimiter
        )

        # Draw the branch connecting the current root node to the left sub-box
        # Pad the line with whitespaces where necessary
        if l_box_width > 0:
            l_root = (l_root_start + l_root_end) // 2 + 1
            line1.append(" " * (l_root + 1))
            line1.append("_" * (l_box_width - l_root))
            line2.append(" " * l_root + "/")
            line2.append(" " * (l_box_width - l_root))
            new_root_start = l_box_width + 1
            gap_size += 1
        else:
            new_root_start = 0

        # Draw the representation of the current root node
        line1.append(node_repr)
        line2.append(" " * new_root_width)

        # Draw the branch connecting the current root node to the right sub-box
        # Pad the line with whitespaces where necessary
        if r_box_width > 0:
            r_root = (r_root_start + r_root_end) // 2
            line1.append("_" * r_root)
            line1.append(" " * (r_box_width - r_root + 1))
            line2.append(" " * r_root + "\\")
            line2.append(" " * (r_box_width - r_root))
            gap_size += 1
        new_root_end = new_root_start + new_root_width - 1

        # Combine the left and right sub-boxes with the branches drawn above
        gap = " " * gap_size
        new_box = ["".join(line1), "".join(line2)]
        for i in range(max(len(l_box), len(r_box))):
            l_line = l_box[i] if i < len(l_box) else " " * l_box_width
            r_line = r_box[i] if i < len(r_box) else " " * r_box_width
            new_box.append(l_line + gap + r_line)

        # Return the new box, its width and its root repr positions
        return new_box, len(new_box[0]), new_root_start, new_root_end

# References
- BST, [Here](https://runestone.academy/ns/books/published//pythonds3/Trees/BinarySearchTrees.html)
- BST Animation, [Here](https://blog.penjee.com/5-gifs-to-understand-binary-search-tree/#binary-search-tree-insertion-node)

!conda install -c conda-forge binarytree

- Home: https://github.com/joowani/binarytree
- Documentation: http://binarytree.readthedocs.io
- https://binarytree.readthedocs.io/en/main/


------
그러므로 이제 그리스도 예수 안에 있는 자에게는 결코 정죄함이 없나니 이는 그리스도 예수 안에 있는 생명의 성령의 법이 죄와 사망의 법에서 너를 해방하였음이라 (롬8:1-2)